먼저, 내가 open3d로 만든 FPFH디스크립터가 어느정도 잘 동작하는지 확인하고 학습해야 할 듯.
내가 잘못 만들어서 문제가 된 것일 수 있으니 확인

In [2]:
import os
import sys
import argparse
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import open3d.visualization as vis
import torch
from scipy.spatial.distance import cdist

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.0,
    seq_list = [0, 2],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    memory_is_enough = True,
    mutual_check = True,
    transform_opt = 0
)

In [4]:
class mdgat_kitti_data_set():
    def __init__(self, opt):
        self.kitti_path = opt.kitti_path
        self.mdgat_path = opt.mdgat_path
        self.seq_list = opt.seq_list
        self.memory_is_enough = opt.memory_is_enough
        self.mutual_check = opt.mutual_check
        self.transform_opt = opt.transform_opt
        self.fpfh_normal_radiuse = opt.fpfh_normal_radiuse
        self.fpfh_descriptors_radiuse = opt.fpfh_descriptors_radiuse

        self.dataset = []
        self.calib = {}
        self.pose = {}
        self.kp = {}
        self.desc = {}
        self.scores = {}
        self.scan = {}
        self.random_sample_num = 16384 #16384
        self.threshold = 0.5

        self._load_kitti_gt_txt()
        self._load_preprocessed_data()

    def _load_kitti_gt_txt(self):
        '''
        :param txt_root:
        :param seq
        :return: [{anc_idx: *, pos_idx: *, seq: *}]                
        '''
        for seq in self.seq_list:
            with open(os.path.join(self.mdgat_path, 'preprocess-random-full','%02d'%seq, 'groundtruths.txt'), 'r') as f:
                lines_list = f.readlines()
                for i, line_str in enumerate(lines_list):
                    if i == 0:
                        # skip the header line
                        continue
                    line_splitted = line_str.split()
                    anc_idx = int(line_splitted[0])
                    pos_idx = int(line_splitted[1])

                    data = {'seq': seq, 'anc_idx': anc_idx, 'pos_idx': pos_idx}
                    self.dataset.append(data)
    
    def _load_preprocessed_data(self):
        for seq in self.seq_list:
            sequence = '%02d'%seq
            calibpath = os.path.join(self.mdgat_path, 'calib/sequences', sequence, 'calib.txt')
            posepath = os.path.join(self.mdgat_path, 'poses', '%02d.txt'%seq)
            with open(calibpath, 'r') as f:
                for line in f.readlines():
                    _, value = line.split(':', 1)
                    try:
                        calib = np.array([float(x) for x in value.split()])
                    except ValueError:
                        pass
                    calib = np.reshape(calib, (3, 4))    
                    self.calib[sequence] = np.vstack([calib, [0, 0, 0, 1]])
            
            poses = []
            with open(posepath, 'r') as f:
                for line in f.readlines():
                    T_w_cam0 = np.fromstring(line, dtype=float, sep=' ')
                    T_w_cam0 = T_w_cam0.reshape(3, 4)
                    T_w_cam0 = np.vstack((T_w_cam0, [0, 0, 0, 1]))
                    poses.append(T_w_cam0)
                self.pose[sequence] = poses

            '''If memory is enough, load all the data'''
            if self.memory_is_enough:
                kps = []
                scores = []
                desc = []
                folder = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence)
                folder = os.listdir(folder)   
                folder.sort(key=lambda x:int(x[:-4]))
                for idx in range(len(folder)):
                    file = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence, folder[idx])
                    if os.path.isfile(file):
                        pc = np.reshape(np.fromfile(file, dtype=np.float32), (-1, 37))
                        ones = np.ones((pc.shape[0], 1))
                        kps.append(np.concatenate((pc[:,:3], ones), axis=1))
                        scores.append(pc[:,3])
                        desc.append(pc[:,4:])
                    else:
                        kps.append([0])

                self.kp[sequence] = kps
                self.desc[sequence] = desc
                self.scores[sequence] = scores

    def _get_kitti_data(self, sequence, index_in_seq):
        pc_file = os.path.join('/media/vision/Seagate/DataSets/kitti/dataset/sequences', sequence, "velodyne" ,'%06d.bin' % index_in_seq)
        pc = np.fromfile(pc_file, dtype=np.float32)
        pc = pc.reshape((-1, 4))

        ones = np.ones((pc.shape[0], 1))
        pc = np.concatenate((pc[:,:3], ones), axis=1)
        # pc = pc[np.random.choice(pc.shape[0], self.random_sample_num, replace=False), :]
        # pc = torch.tensor(pc, dtype=torch.double)
        # pc = pc.reshape((-1, 8))
        return pc
    
    def _get_kitti_poseset(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        pc0 = self._get_kitti_data(sequence, anc_idx)
        pc1 = self._get_kitti_data(sequence, pos_idx)
        return pc0, pc1
    
    def __len__(self):
        return len(self.dataset)
    
    def comute_FPFH(self, pc, kp):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc[:, :3])

        kp_num = kp.shape[0]
        pcd_kp = o3d.geometry.PointCloud()
        pcd_kp.points = o3d.utility.Vector3dVector(kp[:, :3])
        pcd += pcd_kp

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=50))

        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=100))
        fpfh_desc = np.transpose(pcd_fpfh.data)[-kp_num:]
        fpfh_desc = torch.tensor(fpfh_desc, dtype=torch.double)
        return fpfh_desc
    
    def get_data(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        pose0 = torch.tensor(self.pose[sequence][anc_idx], dtype=torch.double)
        pose1 = torch.tensor(self.pose[sequence][pos_idx], dtype=torch.double)

        kp0 = torch.tensor(self.kp[sequence][anc_idx], dtype=torch.double)
        kp1 = torch.tensor(self.kp[sequence][pos_idx], dtype=torch.double)

        desc0 = torch.tensor(self.desc[sequence][anc_idx], dtype=torch.double)
        desc1 = torch.tensor(self.desc[sequence][pos_idx], dtype=torch.double)

        score0 = torch.tensor(self.scores[sequence][anc_idx], dtype=torch.double)
        score1 = torch.tensor(self.scores[sequence][pos_idx], dtype=torch.double)

        pc0, pc1 = self._get_kitti_poseset(idx)
        pc0 = torch.tensor(pc0, dtype=torch.double)
        pc1 = torch.tensor(pc1, dtype=torch.double)

        T_cam0_velo = self.calib[sequence]
        T_cam0_velo = torch.tensor(T_cam0_velo, dtype=torch.double)
        T_gt = torch.einsum('ab,bc,cd,de->ae', torch.inverse(T_cam0_velo), torch.inverse(pose0), pose1, T_cam0_velo) # T_gt: transpose kp2 to kp1

        '''transform point cloud from cam0 to LiDAR'''
        if self.transform_opt == 0:
            kp0w_np = torch.einsum('ki,ij,jm->mk', pose0, T_cam0_velo, kp0.T)
            kp1w_np = torch.einsum('ki,ij,jm->mk', pose1, T_cam0_velo, kp1.T)
            pc0w = torch.einsum('ki,ij,jm->mk', pose0, T_cam0_velo, pc0.T)
            pc1w = torch.einsum('ki,ij,jm->mk', pose1, T_cam0_velo, pc1.T)
            # pc0w = torch.cat((pc0w, kp0w_np), dim=0)
        elif self.transform_opt == 1:
            pc0 = torch.einsum('ki,im->mk', T_cam0_velo, pc0.T)
            pc1 = torch.einsum('ki,im->mk', T_cam0_velo, pc1.T)
            pc0w = torch.einsum('ki,im->mk', pose0, pc0.T)
            pc1w = torch.einsum('ki,im->mk', pose1, pc1.T)
            kp0 = torch.einsum('ki,im->mk', T_cam0_velo, kp0.T)
            kp1 = torch.einsum('ki,im->mk', T_cam0_velo, kp1.T)
            kp0w_np = torch.einsum('ki,im->mk', pose0, kp0.T)
            kp1w_np = torch.einsum('ki,im->mk', pose1, kp1.T)

        fpfh_desc0 = self.comute_FPFH(pc0, kp0)
        fpfh_desc1 = self.comute_FPFH(pc1, kp1)

        '''transform pose from cam0 to LiDAR'''
        # print('pose0 shape: ', pose0.shape, 'T_cam0_velo shape: ', T_cam0_velo.shape, 'kp0 shape: ', kp0.shape)
        kp0w_np = kp0w_np[:, :3]
        kp1w_np = kp1w_np[:, :3]
        dists = cdist(kp0w_np, kp1w_np)
        '''Find ground true keypoint matching'''
        min0 = np.argmin(dists, axis=0)
        min1 = np.argmin(dists, axis=1)
        min0v = np.min(dists, axis=1)
        min0f = min1[min0v < self.threshold]
        '''For calculating repeatibility'''
        rep = len(min0f)
        '''
        If you got high-quality keypoints, you can set the 
        mutual_check to True, otherwise, it is better to 
        set to False
        '''
        match0, match1 = -1 * np.ones((len(kp0)), dtype=np.int16), -1 * np.ones((len(kp1)), dtype=np.int16)
        if self.mutual_check:
            xx = np.where(min1[min0] == np.arange(min0.shape[0]))[0]
            matches = np.intersect1d(min0f, xx)

            match0[min0[matches]] = matches
            match1[matches] = min0[matches]
        else:
            match0[min0v < self.threshold] = min0f

            min1v = np.min(dists, axis=0)
            min1f = min0[min1v < self.threshold]
            match1[min1v < self.threshold] = min1f

        
        norm0, norm1 = np.linalg.norm(desc0, axis=1), np.linalg.norm(desc1, axis=1)
        norm0, norm1 = norm0.reshape(len(kp0), 1), norm1.reshape(len(kp1), 1)
        # desc0, desc1  = np.multiply(desc0, 1/norm0), np.multiply(desc1, 1/norm1)
        desc0, desc1 = np.where(norm0 != 0, np.multiply(desc0, 1/norm0), 0), np.where(norm1 != 0, np.multiply(desc1, 1/norm1), 0)

        fpfh_norm0, fpfh_norm1 = np.linalg.norm(fpfh_desc0, axis=1), np.linalg.norm(fpfh_desc1, axis=1)
        fpfh_norm0, fpfh_norm1 = fpfh_norm0.reshape(len(kp0), 1), fpfh_norm1.reshape(len(kp1), 1)
        fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)
        # fpfh_desc0, fpfh_desc1  = np.multiply(fpfh_desc0, 1/fpfh_norm0), np.multiply(fpfh_desc1, 1/fpfh_norm1)
        
        return {
            # 'skip': False,
            'keypoints0': kp0,
            'keypoints1': kp1,
            'keypointsw0': kp0w_np,
            'keypointsw1': kp1w_np,
            'descriptors0': desc0,
            'descriptors1': desc1,
            'fpfh_descriptors0': fpfh_desc0,
            'fpfh_descriptors1': fpfh_desc1,
            'scores0': score0,
            'scores1': score1,
            'gt_matches0': match0,
            'gt_matches1': match1,
            'sequence': sequence,
            'idx0': anc_idx,
            'idx1': pos_idx,
            'pose0': pose0,
            'pose1': pose1,
            'T_cam0_velo': T_cam0_velo,
            'T_gt': T_gt,
            'cloud0': pc0,
            'cloud1': pc1,
            'cloudw0': pc0w,
            'cloudw1': pc1w,
            # 'all_matches': list(all_matches),
            # 'file_name': file_name
            'rep': rep
        } 

    def __getitem__(self, idx):
        data = self.dataset[idx]
        sequence = '%02d'%data['seq']
        anc_idx = data['anc_idx']
        pos_idx = data['pos_idx']
        anc_pc, pos_pc = self._get_kitti_poseset(idx)
        return anc_pc, pos_pc


[n, 4]크기의 포인트 클라우드의 좌표계를 변환할 때, 마지막 열이 1이 아니라면 변환이 이상하게 될 수 밖에 없다. 따라서 변환해야 하는 경우, I 값을 1로 바꿔주는 것이 중요하다.

In [11]:
# 비주얼라이제이션
data_set = mdgat_kitti_data_set(opt=opt)
data = data_set.get_data(1100)
coord = 1
if coord == 0:
    pc0 = data['cloud0']
    kp0 = data['keypoints0']
    pc1 = data['cloud1']
    kp1 = data['keypoints1']
else:
    pc0 = data['cloudw0']
    kp0 = data['keypointsw0']
    pc1 = data['cloudw1']
    kp1 = data['keypointsw1']

# Create Open3D point cloud objects
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(pc0[:, :3])
pcd0.colors = o3d.utility.Vector3dVector(np.ones((pc0.shape[0], 3)) * [0.5, 0.5, 0.5])

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pc1[:, :3])
pcd1.colors = o3d.utility.Vector3dVector(np.ones((pc1.shape[0], 3)) * [0.2, 0.2, 0.2])

kpcd0 = o3d.geometry.PointCloud()
kpcd0.points = o3d.utility.Vector3dVector(kp0[:, :3])
kpcd0.colors = o3d.utility.Vector3dVector(np.ones((kp0.shape[0], 3)) * [1, 0, 0])

kpcd1 = o3d.geometry.PointCloud()
kpcd1.points = o3d.utility.Vector3dVector(kp1[:, :3])
kpcd1.colors = o3d.utility.Vector3dVector(np.ones((kp1.shape[0], 3)) * [0, 1, 0])

# Visualize the point clouds
vis.draw_geometries([pcd0, kpcd0, pcd1, kpcd1])

/tmp/ipykernel_4770/3345743316.py:215: RuntimeWarning: divide by zero encountered in divide
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)
/tmp/ipykernel_4770/3345743316.py:215: RuntimeWarning: invalid value encountered in multiply
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)


KeyboardInterrupt: 

만들어진 fpfh값과 precomputed된 fpfh값을 비교해서 어떤 세팅값이 가장 유사한지 찾아보자.

In [6]:
test_fpfh_normal_radiuses = [0.2, 0.5, 0.8, 1.0, 1.3, 1.5, 1.7]

# test_fpfh_descriptors_radiuses = [0.7 ,1.0, 1.5, 2.0, 2.5]
test_fpfh_descriptors_radiuses = [1.5]

matched_points_rank_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))
matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))

for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):
        opt = argparse.Namespace(
            fpfh_normal_radiuse = fpfh_normal_radiuse,
            fpfh_descriptors_radiuse = fpfh_descriptors_radiuses,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)

        n = []

        rank_sum = 0
        dist_sum = 0
        ranks = []

        for i in range(100, len(data_set), 500):
            data = data_set.get_data(i)
            match0 = data['gt_matches0']
            desc0 = data['fpfh_descriptors0']
            desc1 = data['fpfh_descriptors1']

            nn = 0
            for idx, matched_idx in enumerate(match0):
                nn += 1
                
                if matched_idx != -1:
                    distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
                    dist_sum += distance[idx]
                    rank_sum += np.argsort(distance).tolist().index(idx)
                    ranks.append(np.argsort(distance).tolist().index(idx))
            n.append(nn)

        matched_points_rank_mean[j,k] = rank_sum / sum(n)
        matched_pointe_dists_mean[j,k] = dist_sum / sum(n)

        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'rank mean: ', matched_points_rank_mean[j,k], 'dist mean: ', matched_pointe_dists_mean[j,k])
        print('ranks: ', ranks)

/tmp/ipykernel_4770/3345743316.py:215: RuntimeWarning: divide by zero encountered in divide
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)
/tmp/ipykernel_4770/3345743316.py:215: RuntimeWarning: invalid value encountered in multiply
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)


radius: ( 0.2 ,  1.5 )  rank mean:  19.726768092105264 dist mean:  0.1420799581731875
ranks:  [222, 146, 144, 229, 174, 146, 44, 44, 244, 115, 163, 210, 200, 110, 0, 121, 206, 112, 116, 0, 76, 28, 131, 24, 226, 249, 2, 173, 11, 223, 32, 50, 135, 4, 67, 75, 72, 241, 158, 44, 253, 49, 95, 122, 9, 109, 197, 86, 54, 11, 63, 52, 193, 247, 160, 63, 137, 2, 123, 3, 183, 40, 50, 59, 37, 9, 94, 1, 102, 232, 80, 188, 3, 8, 173, 2, 92, 1, 184, 39, 118, 183, 31, 237, 30, 17, 9, 50, 113, 38, 247, 59, 198, 228, 19, 10, 217, 153, 223, 48, 46, 45, 221, 91, 40, 150, 154, 95, 6, 8, 16, 93, 95, 81, 12, 82, 98, 27, 214, 188, 133, 24, 181, 226, 208, 254, 36, 91, 15, 46, 31, 38, 94, 171, 73, 216, 96, 210, 75, 18, 194, 221, 116, 34, 187, 70, 207, 21, 11, 45, 154, 35, 29, 152, 101, 13, 40, 17, 2, 13, 11, 60, 144, 176, 61, 214, 189, 40, 41, 13, 132, 247, 224, 39, 61, 9, 18, 78, 182, 45, 110, 107, 4, 8, 15, 37, 115, 10, 249, 205, 223, 208, 246, 75, 0, 210, 11, 26, 95, 253, 121, 51, 85, 79, 141, 131, 68, 131, 18

만들어진 fpfh가 얼마나 잘 describe 하는지 최적의 세팅값을 찾아보자.

In [6]:
test_fpfh_normal_radiuses = [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0, 1.3, 1.5]
test_fpfh_descriptors_radiuses = [0.3, 0.5, 0.7 ,1.0, 1.5, 2.0]

matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))

for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):
        opt = argparse.Namespace(
            fpfh_normal_radiuse = fpfh_normal_radiuse,
            fpfh_descriptors_radiuse = fpfh_descriptors_radiuses,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)

        n = []

        rank_sum = 0
        dist_sum = 0

        for i in range(100, len(data_set), 500):
            data = data_set.get_data(i)
            match0 = data['gt_matches0']
            desc0 = data['fpfh_descriptors0']
            desc1 = data['descriptors0']

            nn = 0
            
            for idx in range(len(desc0)):
                nn += 1
                distance = np.linalg.norm(desc0[idx] - desc1[idx])
                dist_sum += distance
            n.append(nn)

        matched_pointe_dists_mean[j,k] = dist_sum / sum(n)

        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'dist mean: ', matched_pointe_dists_mean[j,k])

/tmp/ipykernel_2422059/3345743316.py:215: RuntimeWarning: divide by zero encountered in divide
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)
/tmp/ipykernel_2422059/3345743316.py:215: RuntimeWarning: invalid value encountered in multiply
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)


radius: ( 0.05 ,  0.3 )  dist mean:  0.9979383225037233
radius: ( 0.05 ,  0.5 )  dist mean:  0.9979383225037233
radius: ( 0.05 ,  0.7 )  dist mean:  0.9979383225037233
radius: ( 0.05 ,  1.0 )  dist mean:  0.9979383225037233
radius: ( 0.05 ,  1.5 )  dist mean:  0.9979383225037233
radius: ( 0.05 ,  2.0 )  dist mean:  0.9979383225037233
radius: ( 0.1 ,  0.3 )  dist mean:  0.9761229104732453
radius: ( 0.1 ,  0.5 )  dist mean:  0.9761229104732453
radius: ( 0.1 ,  0.7 )  dist mean:  0.9761229104732453
radius: ( 0.1 ,  1.0 )  dist mean:  0.9761229104732453
radius: ( 0.1 ,  1.5 )  dist mean:  0.9761229104732453
radius: ( 0.1 ,  2.0 )  dist mean:  0.9761229104732453
radius: ( 0.2 ,  0.3 )  dist mean:  0.8681327103207106
radius: ( 0.2 ,  0.5 )  dist mean:  0.8681327103207106
radius: ( 0.2 ,  0.7 )  dist mean:  0.8681327103207106
radius: ( 0.2 ,  1.0 )  dist mean:  0.8681327103207106
radius: ( 0.2 ,  1.5 )  dist mean:  0.8681327103207106
radius: ( 0.2 ,  2.0 )  dist mean:  0.8681327103207106
radi

In [7]:
data = data_set.get_data(1100)
# print(data['rep'])
match0 = data['gt_matches0']
match1 = data['gt_matches1']
# desc00 = data['descriptors0']
# desc1 = data['descriptors1']
desc0 = data['fpfh_descriptors0']
desc1 = data['fpfh_descriptors1']
kp0 = data['keypointsw0']
kp1 = data['keypointsw1']
for idx, matched_idx in enumerate(match0):
    if matched_idx != -1:
        print('matched_idx: ', idx, ' & ', matched_idx, ' || distance: ', np.linalg.norm(kp0[idx] - kp1[matched_idx]))
        distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
        print('distance with matched points: ', distance[idx], ' || similarity rank: ', np.argsort(distance).tolist().index(idx), '/', len(distance))
        print('min distance: ', np.min(distance))
        print('mean distance: ', np.mean(distance))
        print('----------------------------------')

matched_idx:  3  &  9  || distance:  0.18763622760874527
distance with matched points:  0.33330476476521226  || similarity rank:  0 / 256
min distance:  0.33330476476521226
mean distance:  0.8610925759486717
----------------------------------
matched_idx:  5  &  57  || distance:  0.26162182280272883
distance with matched points:  0.21412167942389143  || similarity rank:  1 / 256
min distance:  0.1984820485056124
mean distance:  0.6708701055599533
----------------------------------
matched_idx:  7  &  96  || distance:  0.4986982784203404
distance with matched points:  0.5457079875855159  || similarity rank:  58 / 256
min distance:  0.3278648910766215
mean distance:  0.6791749320283037
----------------------------------
matched_idx:  10  &  104  || distance:  0.10625041912095994
distance with matched points:  0.4174932776960585  || similarity rank:  2 / 256
min distance:  0.398312577175682
mean distance:  0.7077140237388004
----------------------------------
matched_idx:  11  &  77  || d

In [5]:
from scipy.spatial import cKDTree

def extract_fpfh(pcd, voxel_size):
  radius_normal = voxel_size * 2
  pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

  radius_feature = voxel_size * 5
  fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
  return np.array(fpfh.data).T

def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
    feat1tree = cKDTree(feat1)
    dists, nn_inds = feat1tree.query(feat0, k=knn)
    if return_distance:
        return nn_inds, dists
    else:
        return nn_inds
    
def find_correspondences1(feats0, feats1):
    dists = cdist(feats0, feats1)

    '''Find ground true keypoint matching'''
    min1 = np.argmin(dists, axis=0)
    min2 = np.argmin(dists, axis=1)
    min1v = np.min(dists, axis=1)
    min1f = min2[min1v < 0.5]

    match1, match2 = -2 * np.ones((len(feats0)), dtype=np.int16), -2 * np.ones((len(feats1)), dtype=np.int16)
    xx = np.where(min2[min1] == np.arange(min1.shape[0]))[0]
    matches = np.intersect1d(min1f, xx)

    match1[min1[matches]] = matches
    match2[matches] = min1[matches]

    return match1, match2

def find_correspondences(feats0, feats1, mutual_filter=True):
    """
    Find correspondences between two sets of features.

    Args:
        feats0 (numpy.ndarray): Array of features from the first set.
        feats1 (numpy.ndarray): Array of features from the second set.
        mutual_filter (bool, optional): Whether to apply mutual filtering. Defaults to True.

    Returns:
        tuple: A tuple containing two numpy arrays: corres_idx0 and corres_idx1.
               corres_idx0 (numpy.ndarray): Array of indices of correspondences in feats0.
               corres_idx1 (numpy.ndarray): Array of indices of correspondences in feats1.
    """
    # print(feats0.shape, feats1.shape)
    nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
    # print(nns01.shape)
    corres01_idx0 = np.arange(len(nns01))
    corres01_idx1 = nns01

    if not mutual_filter:
        return corres01_idx0, corres01_idx1

    nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
    corres10_idx1 = np.arange(len(nns10))
    corres10_idx0 = nns10

    # print(corres01_idx0, corres01_idx1)

    match1, match2 = -1 * np.ones((len(feats0)), dtype=np.int16), -1 * np.ones((len(feats1)), dtype=np.int16)

    mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
    # print(mutual_filter)
    corres01_idx0[~mutual_filter] = -2
    corres01_idx1[~mutual_filter] = -2
    # print(corres01_idx0)
    
    # corres_idx0 = corres01_idx0[mutual_filter]
    # print(corres_idx0)
    # corres_idx1 = corres01_idx1[mutual_filter]

    return corres01_idx0, corres01_idx1

In [13]:
test_fpfh_normal_radiuses = [0.4, 0.5, 0.6, 0.7]

test_fpfh_descriptors_radiuses = [0.7 ,1.0, 1.5]
# test_fpfh_descriptors_radiuses = [1.5]

matched_points_rank_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))
matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))
correct_list = []
for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):   
        opt = argparse.Namespace(
            fpfh_normal_radiuse = j,
            fpfh_descriptors_radiuse = k,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)
        correct = 0
        for i in range(0, len(data_set), 500):
            data = data_set.get_data(i)
            pc0 = data['cloud0']
            kp0 = data['keypoints0']
            pc1 = data['cloud1']
            kp1 = data['keypoints1']
            desc0 = data['descriptors0']
            desc1 = data['descriptors1']
            fpfh_desc0 = data['fpfh_descriptors0']
            fpfh_desc1 = data['fpfh_descriptors1']
            match0 = data['gt_matches0']
            match1 = data['gt_matches1']

            pcd0 = o3d.geometry.PointCloud()
            pcd0.points = o3d.utility.Vector3dVector(kp0[:, :3])
            pcd1 = o3d.geometry.PointCloud()
            pcd1.points = o3d.utility.Vector3dVector(kp1[:, :3])

            pcd0_np = pcd2xyz(pcd0) # np array of size 3 by N
            pcd1_np = pcd2xyz(pcd1) # np array of size 3 by M

            corrs_pcd0, corrs_pcd1 = find_correspondences1(fpfh_desc0, fpfh_desc1)
            # print(sum(corrs_pcd0>-1))

            # print(corrs_pcd0, corrs_pcd1)
            pcd0_corr = pcd0_np[:,corrs_pcd0>-1] # np array of size 3 by num_corrs
            pcd1_corr = pcd1_np[:,corrs_pcd1>-1] # np array of size 3 by num_corrs

            num_corrs = pcd0_corr.shape[1]
            print(f'FPFH generates {num_corrs} putative correspondences.', end=' ')
            print(f'gt has {sum(match0>-1)} correspondences.')

            points = np.concatenate((pcd0_corr.T,pcd1_corr.T),axis=0)
            lines = []
            for i in range(num_corrs):
                lines.append([i,i+num_corrs])
            colors = [[0, 1, 0] for i in range(len(lines))] # lines are shown in green
            line_set = o3d.geometry.LineSet(
                points=o3d.utility.Vector3dVector(points),
                lines=o3d.utility.Vector2iVector(lines),
            )
            line_set.colors = o3d.utility.Vector3dVector(colors)
            # o3d.visualization.draw_geometries([pcd0,pcd1,line_set])

            # print(match0, corrs_pcd0)
            correct += sum(match0 == corrs_pcd0)
        correct_list.append(correct)
        # print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'successed match: ', correct)
        # print(f'successed match: {sum(correct)}')
i = 0
for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):   
        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'successed match: ', correct_list[i])
        i+=1

/tmp/ipykernel_23238/3144579386.py:216: RuntimeWarning: divide by zero encountered in divide
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)
/tmp/ipykernel_23238/3144579386.py:216: RuntimeWarning: invalid value encountered in multiply
  fpfh_desc0, fpfh_desc1 = np.where(fpfh_norm0 != 0, np.multiply(fpfh_desc0, 1/fpfh_norm0), 0), np.where(fpfh_norm1 != 0, np.multiply(fpfh_desc1, 1/fpfh_norm1), 0)


FPFH generates 1 putative correspondences. gt has 60 correspondences.
FPFH generates 1 putative correspondences. gt has 29 correspondences.
FPFH generates 1 putative correspondences. gt has 64 correspondences.
FPFH generates 1 putative correspondences. gt has 64 correspondences.
FPFH generates 1 putative correspondences. gt has 88 correspondences.
FPFH generates 1 putative correspondences. gt has 72 correspondences.
FPFH generates 1 putative correspondences. gt has 23 correspondences.
FPFH generates 1 putative correspondences. gt has 74 correspondences.
FPFH generates 1 putative correspondences. gt has 10 correspondences.
FPFH generates 1 putative correspondences. gt has 59 correspondences.
FPFH generates 1 putative correspondences. gt has 63 correspondences.
FPFH generates 1 putative correspondences. gt has 23 correspondences.
FPFH generates 1 putative correspondences. gt has 33 correspondences.
FPFH generates 1 putative correspondences. gt has 13 correspondences.
FPFH generates 1 put

In [12]:

i = 0
for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):   
        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'successed match: ', correct_list[i])
        i+=1

radius: ( 0.2 ,  0.7 )  successed match:  0
radius: ( 0.2 ,  1.0 )  successed match:  0
radius: ( 0.2 ,  1.5 )  successed match:  0
radius: ( 0.2 ,  2.0 )  successed match:  0
radius: ( 0.2 ,  2.5 )  successed match:  0
radius: ( 0.5 ,  0.7 )  successed match:  50
radius: ( 0.5 ,  1.0 )  successed match:  50
radius: ( 0.5 ,  1.5 )  successed match:  50
radius: ( 0.5 ,  2.0 )  successed match:  50
radius: ( 0.5 ,  2.5 )  successed match:  50
radius: ( 0.8 ,  0.7 )  successed match:  46
radius: ( 0.8 ,  1.0 )  successed match:  46
radius: ( 0.8 ,  1.5 )  successed match:  46
radius: ( 0.8 ,  2.0 )  successed match:  46
radius: ( 0.8 ,  2.5 )  successed match:  46
radius: ( 1.0 ,  0.7 )  successed match:  47
radius: ( 1.0 ,  1.0 )  successed match:  47
radius: ( 1.0 ,  1.5 )  successed match:  47
radius: ( 1.0 ,  2.0 )  successed match:  47
radius: ( 1.0 ,  2.5 )  successed match:  47
radius: ( 1.3 ,  0.7 )  successed match:  47
radius: ( 1.3 ,  1.0 )  successed match:  47
radius: ( 1.3 ,